<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/MLP/wine_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch


In [19]:
#preparing dataset
wine = load_wine()
# x is the carachteristics and y the labels
x = wine.data
y = wine.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Padronizar os dados para média zero e variância unitária, melhora o treinamento
#transforming trains and tests munpy arrays to tensors
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # Padroniza os dados de treino
x_test = scaler.transform(x_test)  # Padroniza os dados de teste (usando os mesmos parâmetros do treino)

# Converter para tensores do PyTorch
x_train= torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [20]:
#defining model
#herda de torch.nn.Module
class MLP(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    #chama superclasse
    super(MLP, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)
    #define o comportamento da rede neural
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x



In [4]:
print(x_train.size())
print(x_test.size())

torch.Size([142, 13])
torch.Size([36, 13])


In [21]:
model = MLP(input_size=x_train.size()[1], hidden_size=30, output_size=3)


#defining loss function
criterion = torch.nn.CrossEntropyLoss()
#defining optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.009)

epochs = 100

for epoch in range(epochs):
  model.train()
  #always remember this!!
  optimizer.zero_grad()

  outputs = model(x_train)
  loss = criterion(outputs, y_train)

  #calculating gradients
  loss.backward()
  #updating weights
  optimizer.step()
  if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.2585
Epoch [20/100], Loss: 0.0197
Epoch [30/100], Loss: 0.0018
Epoch [40/100], Loss: 0.0004
Epoch [50/100], Loss: 0.0002
Epoch [60/100], Loss: 0.0001
Epoch [70/100], Loss: 0.0001
Epoch [80/100], Loss: 0.0001
Epoch [90/100], Loss: 0.0001
Epoch [100/100], Loss: 0.0001


In [22]:
#evaluating model
#no need to calculate gradient in evaluation
with torch.no_grad():
  outputs = model(x_test)
  _, predicted = torch.max(outputs.data, 1)
  accuracy = (predicted == y_test).sum().item() / len(y_test)
  print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 100.00%
